In [ ]:
import pandas as pd
from sklearn.metrics import f1_score
import numpy as np

In [ ]:
pd.set_option("max_colwidth", 100)
pd.set_option("display.max_rows",10)
pd.set_option("display.max_columns",5)
DS= pd.read_pickle('balanced_movies_genre.pkl');
genre_data = DS
genre_data

In [ ]:
!pip install --upgrade openai

In [ ]:
from openai import OpenAI
client = OpenAI(
    api_key='your api key',
)

In [ ]:
def get_completion(messages, model="gpt-4o"):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

In [ ]:
def get_completion(messages, model="gpt-3.5-turbo-0125"):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

In [ ]:

import time

def connect_with_api(messages):
    max_retries = 3
    retries = 0

    while retries < max_retries:
        try:
            response = get_completion(messages)
            return response
        except Exception as e:
            print(f"An error occurred: {e}")
            retries += 1
            print(f"Retrying... (Attempt {retries} of {max_retries})")
            time.sleep(1)  # Wait for 1 second before retrying

    print("Failed to connect with the API after multiple attempts.")

In [ ]:
test_samples_per_genre = 500
train_samples_per_genre = 5
test_df = pd.DataFrame()
train_df = pd.DataFrame()
for genre in genre_data['Genre'].unique():
    genre_df = genre_data[genre_data['Genre'] == genre]
    test_samples = genre_df.sample(test_samples_per_genre, random_state=1)
    train_samples = genre_df.drop(test_samples.index).sample(train_samples_per_genre, random_state=1)

    test_df = pd.concat([test_df, test_samples])
    train_df = pd.concat([train_df, train_samples])

# Reset the index of the resulting DataFrames
test_df = test_df.reset_index(drop=True)
train_df = train_df.reset_index(drop=True)

In [ ]:
def clean_response(response):
    response = response.strip().lower()
    if 'comedy' in response:
        return 'comedy'
    elif 'action' in response:
        return 'action'
    elif 'drama' in response:
        return 'drama'
    elif 'horror' in response:
        return 'horror'
    else:
        return None

In [ ]:

# Experiment configurations
def run_experiment(prompt_design, test_df, train_df, experiment_name):
    test_results = []
    for index, row in test_df.iterrows():
        if prompt_design == "zeroU":
            messages = [{"role": "user", "content": f"""Determine the genre of the movie based on the provided plot:
             For the plot provided, classify its genre as a single word (without other marks or words like 'genre:'), either "comedy", "action", "drama", or "horror".
```{row['Plot']}```""" }]
        elif prompt_design == "zeroSU":
            messages = [{"role": "system", "content": f"""Determine the genre of the movie based on the provided plot.
             For the plot provided, classify its genre as a single word (without other marks or words like 'genre:'), either "comedy", "action", "drama", or "horror"."""},
                        {"role": "user", "content": f"""```{row['Plot']}```"""}]
        elif prompt_design == "fewU":
            messages = [{"role": "user", "content": f"""Determine the genre of the movie based on the provided plot:
             For the plot provided, classify its genre as a single word (without other marks or words like 'genre:'), either "comedy", "action", "drama", or "horror".
Examples:
```{train_df.iloc[0]['Plot']}``` - {train_df.iloc[0]['Genre']}
```{train_df.iloc[14]['Plot']}``` - {train_df.iloc[14]['Genre']}
```{train_df.iloc[17]['Plot']}``` - {train_df.iloc[17]['Genre']}
...
```{row['Plot']}```""" }]
        elif prompt_design == "fewSU":
            messages = [{"role": "system", "content": f"""Determine the genre of the movie based on the provided plot.
             For the plot provided, classify its genre as a single word (without other marks or words like 'genre:'), either "comedy", "action", "drama", or "horror"."""},
                        {"role": "user", "content": f"""Examples:
```{train_df.iloc[0]['Plot']}``` - {train_df.iloc[0]['Genre']}
```{train_df.iloc[14]['Plot']}``` - {train_df.iloc[14]['Genre']}
```{train_df.iloc[17]['Plot']}``` - {train_df.iloc[17]['Genre']}
...
```{row['Plot']}```"""}]
        elif prompt_design == "fewSUA":
            messages = [{"role": "system", "content": f"""Determine the genre of the movie based on the provided plot.
            For the plot provided, classify its genre as a single word (without other marks or words like 'genre:'), either "comedy", "action", "drama", or "horror"."""},
                        {"role": "user", "content": train_df.iloc[0]['Plot']}, {"role": "assistant", "content": train_df.iloc[0]['Genre']},
                        {"role": "user", "content": train_df.iloc[14]['Plot']}, {"role": "assistant", "content": train_df.iloc[14]['Genre']},
                        {"role": "user", "content": train_df.iloc[17]['Plot']}, {"role": "assistant", "content": train_df.iloc[17]['Genre']},
                        {"role": "user", "content": f"""```{row['Plot']}```"""}]

        prediction = connect_with_api(messages)
        print(index, ' ', prediction)

        test_results.append({'Index': index, 'Plot': row['Plot'], 'Predicted_Genre': prediction, 'Actual_Genre': row['Genre']})

    results_df = pd.DataFrame(test_results)
    # Calculate structural accuracy before cleaning
    valid_genres = ['comedy', 'action', 'drama', 'horror']
    results_df['Is_Structurally_Valid'] = results_df['Predicted_Genre'].apply(lambda x: x.strip().lower() in valid_genres)
    structural_accuracy = results_df['Is_Structurally_Valid'].mean()

    # Calculate F1 Score
    results_df['Predicted_Genre'] = results_df['Predicted_Genre'].apply(clean_response)
    results_df = results_df.dropna(subset=['Predicted_Genre', 'Actual_Genre'])
    f1 = f1_score(results_df['Actual_Genre'], results_df['Predicted_Genre'], average='micro')

    print(f"Experiment: {experiment_name}")
    print(f"The F1 Score for the predicted genre is: {f1}")
    print(f"The Structural Accuracy is: {structural_accuracy}")

    results_df.to_csv(f"/content/drive/MyDrive/Colab Notebooks/In_context_learning/{experiment_name}_results.csv", index=False)
    print(f"Results for {experiment_name} saved to {experiment_name}_results.csv")

In [ ]:
run_experiment("zeroU", test_df, train_df, "Zero-shot User Prompt")


In [ ]:
run_experiment("zeroSU", test_df, train_df, "Zero-shot System and User Prompt")

In [ ]:
run_experiment("fewU", test_df, train_df, "Few-shot User Prompt")


In [ ]:
run_experiment("fewSU", test_df, train_df, "Few-shot System and User Prompt")


In [ ]:
run_experiment("fewSUA", test_df, train_df, "Few-shot System, User, and Assistant Prompt")